# Broadcasting and Advanced NumPy Manipulation
Programación II — Semana 2

*Fecha de generación: 2025-05-20*

## Objetivos de aprendizaje
Al finalizar este notebook podrás:
1. Explicar la **regla de broadcasting** y aplicarla a operaciones vectorizadas.
2. Utilizar `reshape`, `concatenate`, `split` y vistas para reorganizar datos sin copiar memoria.
3. Medir el impacto de **vistas** vs **copias** en rendimiento y uso de memoria.
4. Resolver problemas prácticos de procesamiento NumPy sin bucles explícitos.


## 1 · Broadcasting en NumPy  
Cuando operamos con arrays de distinta forma, NumPy **estira** las dimensiones de tamaño `1` para hacerlas compatibles.  
**Regla:** se comparan shapes de **derecha a izquierda**.

![](https://numpy.org/doc/stable/_images/broadcast.svg)



In [1]:
import numpy as np

A = np.arange(6).reshape(3,2)        # (3,2)
B = np.array([10, 100])              # (2,)
       # B se transmite sobre la primera dimensión


In [2]:
print(A)
print(B)

[[0 1]
 [2 3]
 [4 5]]
[ 10 100]


In [3]:
print("A =\n", A)
print("B =", B)
print("A + B =\n", A + B)    

A =
 [[0 1]
 [2 3]
 [4 5]]
B = [ 10 100]
A + B =
 [[ 10 101]
 [ 12 103]
 [ 14 105]]


### 📝 Ejercicio 1  
Convierte un vector `temp_f` de temperaturas en °F a °C usando broadcasting y la fórmula  

$$
T_C = (T_F - 32) \times \frac{5}{9} $$

*Sin usar bucles ni comprensiones de listas.*

In [4]:


temp_f = np.array([32, 68, 95, 104])  # ejemplo
print("temp_f =", temp_f)


temp_f = [ 32  68  95 104]


In [5]:
temp_c= (temp_f - 32) * 5/9

print("temp_c =", temp_c)


temp_c = [ 0. 20. 35. 40.]


### 📝 Ejercicio 2  
Dado un conjunto `P` de `n` puntos 2‑D y otro `Q` de `m` puntos 2‑D, calcula la matriz de distancias euclidianas $$ (D\in\mathbb{R}^{n\times m}) $$ **sin usar bucles** (pista: broadcasting y sumas sobre ejes).

In [7]:
import numpy as np

# Generar datos de ejemplo
# P: n puntos, cada uno con 2 coordenadas (n x 2)
n_puntos_P = 5
P = np.random.rand(n_puntos_P, 2) * 10 # Puntos entre 0 y 10

# Q: m puntos, cada uno con 2 coordenadas (m x 2)
m_puntos_Q = 3
Q = np.random.rand(m_puntos_Q, 2) * 10 # Puntos entre 0 y 10

print("P ({} puntos):\n".format(n_puntos_P), P)
print("Q ({} puntos):\n".format(m_puntos_Q), Q)

# Calcular la matriz de distancias euclidianas D (n x m)

P_expandido = P[:, np.newaxis, :] # Forma: (n, 1, 2)
Q_expandido = Q[np.newaxis, :, :] # Forma: (1, m, 2)

# Calcular las diferencias al cuadrado para cada coordenada

diferencias_cuadrado = (P_expandido - Q_expandido)**2

# Sumar las diferencias al cuadrado a lo largo del eje de las coordenadas (eje 2)
suma_diferencias_cuadrado = np.sum(diferencias_cuadrado, axis=2)

# Calcular la raíz cuadrada para obtener la distancia euclidiana
D = np.sqrt(suma_diferencias_cuadrado)

print("\nMatriz de distancias D ({}x{}):\n".format(n_puntos_P, m_puntos_Q), D)
print("Forma de D:", D.shape)

# Verificación para un par de puntos (opcional)
# Distancia entre P[0] y Q[0]
dist_manual_00 = np.sqrt(np.sum((P[0] - Q[0])**2))
print(f"\nDistancia manual entre P[0] y Q[0]: {dist_manual_00:.4f}")
print(f"Distancia calculada D[0,0]: {D[0,0]:.4f}")

P (5 puntos):
 [[2.49292229 4.10382923]
 [7.55551139 2.28798165]
 [0.7697991  2.89751453]
 [1.61221287 9.29697652]
 [8.0812038  6.33403757]]
Q (3 puntos):
 [[8.7146059  8.03672077]
 [1.86570059 8.92558998]
 [5.39342242 8.07440155]]

Matriz de distancias D (5x3):
 [[7.36050153 4.86238458 4.91714811]
 [5.86442678 8.74252774 6.17715824]
 [9.46210314 6.12688288 6.94104112]
 [7.21333704 0.44964873 3.97394454]
 [1.81668058 6.7341387  3.20203615]]
Forma de D: (5, 3)

Distancia manual entre P[0] y Q[0]: 7.3605
Distancia calculada D[0,0]: 7.3605


In [7]:
P_corrected=P[:,np.newaxis,:]
print("P_corrected =\n", P_corrected)
print("P_corrected.shape =", P_corrected.shape)

P_corrected =
 [[[0 6]]

 [[0 0]]

 [[1 8]]

 [[0 1]]

 [[6 3]]]
P_corrected.shape = (5, 1, 2)


In [8]:
Q_corrected= Q[np.newaxis,:,:]
print("Q_corrected =\n", Q_corrected)   
print("Q_corrected.shape =", Q_corrected.shape) 

Q_corrected =
 [[[6 7]
  [2 0]
  [8 5]]]
Q_corrected.shape = (1, 3, 2)


In [9]:
# Calcula la matriz de distancias euclidianas entre P (5,2) y Q (3,2)
diff = P_corrected-Q_corrected # (5,3,2)
print("diff =\n", diff)

diff =
 [[[-6 -1]
  [-2  6]
  [-8  1]]

 [[-6 -7]
  [-2  0]
  [-8 -5]]

 [[-5  1]
  [-1  8]
  [-7  3]]

 [[-6 -6]
  [-2  1]
  [-8 -4]]

 [[ 0 -4]
  [ 4  3]
  [-2 -2]]]


In [10]:
D= np.linalg.norm(diff, axis=2) # (5,3)
print("D =\n", D)
print("D.shape =", D.shape)

D =
 [[6.08276253 6.32455532 8.06225775]
 [9.21954446 2.         9.43398113]
 [5.09901951 8.06225775 7.61577311]
 [8.48528137 2.23606798 8.94427191]
 [4.         5.         2.82842712]]
D.shape = (5, 3)


In [11]:
print(P)
print(Q)

[[0 6]
 [0 0]
 [1 8]
 [0 1]
 [6 3]]
[[6 7]
 [2 0]
 [8 5]]


## 2 · Manipulación avanzada de arrays

In [13]:
vec = np.arange(24)
tensor = vec.reshape(2,3,4)   # O(1), sin copiar
print("tensor shape:", tensor.shape)
flat = tensor.ravel()         # vista
print("¿Comparte memoria?", flat.base is tensor)

print("tensor =\n", tensor) 
print("flat =\n", flat)


tensor shape: (2, 3, 4)
¿Comparte memoria? False
tensor =
 [[[ 0  1  2  3]
  [ 4  5  6  7]
  [ 8  9 10 11]]

 [[12 13 14 15]
  [16 17 18 19]
  [20 21 22 23]]]
flat =
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]


### 📝 Ejercicio 3  
Reestructura `tensor` para que sus ejes queden con forma `(3,2,4)` **sin copiar** datos. Comprueba que la vista comparte memoria con el original.

In [6]:
import numpy as np

# Creamos el tensor original como se indica en el contexto del notebook
vec = np.arange(24)
tensor = vec.reshape(2,3,4) # Forma original (2,3,4)

print("Tensor original (forma {}):\n".format(tensor.shape), tensor)

# Para reestructurar de (2,3,4) a (3,2,4), necesitamos intercambiar los ejes 0 y 1.
# Usamos np.swapaxes() o tensor.transpose() que devuelven vistas.
tensor_swapped = tensor.transpose(1, 0, 2)
# Alternativamente:
# tensor_swapped = np.swapaxes(tensor, 0, 1)

print("\nTensor con ejes intercambiados (forma {}):\n".format(tensor_swapped.shape), tensor_swapped)

# Comprobar si comparte memoria
# Si tensor_swapped es una vista de tensor, tensor_swapped.base será tensor
comparte_memoria = tensor_swapped.base is tensor

print("\n¿El tensor intercambiado comparte memoria con el original?:", comparte_memoria)

# Para estar seguros, modificamos un elemento en la vista y vemos si cambia en el original
if comparte_memoria:
    valor_original_primer_elemento = tensor[0,0,0].copy() # Copiamos para no modificarlo antes de la prueba
    tensor_swapped[0,0,0] = 999 # Modificamos el primer elemento de la vista que corresponde a tensor[0,0,0]
    print(f"Valor original de tensor[0,0,0]: {valor_original_primer_elemento}")
    print(f"Valor de tensor[0,0,0] después de modificar tensor_swapped[0,0,0]: {tensor[0,0,0]}")
    # Restauramos el valor para no afectar otros posibles usos del tensor
    tensor[0,0,0] = valor_original_primer_elemento

Tensor original (forma (2, 3, 4)):
 [[[ 0  1  2  3]
  [ 4  5  6  7]
  [ 8  9 10 11]]

 [[12 13 14 15]
  [16 17 18 19]
  [20 21 22 23]]]

Tensor con ejes intercambiados (forma (3, 2, 4)):
 [[[ 0  1  2  3]
  [12 13 14 15]]

 [[ 4  5  6  7]
  [16 17 18 19]]

 [[ 8  9 10 11]
  [20 21 22 23]]]

¿El tensor intercambiado comparte memoria con el original?: False


### 📝 Ejercicio 4  
Carga el dataset *Iris* disponible en NumPy (`np.loadtxt` o `sklearn.datasets.load_iris`) y  
divide las 150 observaciones en un **80 % train / 20 % test** usando `np.split` (sin librerías externas).

In [1]:
import numpy as np
from sklearn.datasets import load_iris

# Cargar el dataset Iris
iris = load_iris()
data = iris.data # Matriz de características (150 muestras, 4 características)

# Número total de observaciones
n_total_observaciones = data.shape[0] # Debería ser 150

# Calcular el número de observaciones para el conjunto de entrenamiento (80%)
n_train = int(n_total_observaciones * 0.8)

# Es buena idea barajar los datos antes de dividirlos para evitar sesgos
# si los datos están ordenados de alguna manera (ej. por clase)
np.random.seed(42) # Para reproducibilidad
indices_barajados = np.random.permutation(n_total_observaciones)
data_barajada = data[indices_barajados]

# Dividir los datos usando np.split
# np.split toma una lista de índices donde se realizarán las divisiones.
# Para dividir en dos partes (train y test), necesitamos un solo índice de división.
train_set, test_set = np.split(data_barajada, [n_train])

print("Forma del dataset original:", data.shape)
print("Número de muestras para entrenamiento (80%):", n_train)
print("Número de muestras para prueba (20%):", n_total_observaciones - n_train)
print("Forma del conjunto de entrenamiento:", train_set.shape)
print("Forma del conjunto de prueba:", test_set.shape)

# Primeras 5 filas del conjunto de entrenamiento
print("\nPrimeras 5 filas del conjunto de entrenamiento:\n", train_set[:5])
# Primeras 5 filas del conjunto de prueba
print("\nPrimeras 5 filas del conjunto de prueba:\n", test_set[:5])


Forma del dataset original: (150, 4)
Número de muestras para entrenamiento (80%): 120
Número de muestras para prueba (20%): 30
Forma del conjunto de entrenamiento: (120, 4)
Forma del conjunto de prueba: (30, 4)

Primeras 5 filas del conjunto de entrenamiento:
 [[6.1 2.8 4.7 1.2]
 [5.7 3.8 1.7 0.3]
 [7.7 2.6 6.9 2.3]
 [6.  2.9 4.5 1.5]
 [6.8 2.8 4.8 1.4]]

Primeras 5 filas del conjunto de prueba:
 [[6.1 3.  4.6 1.4]
 [4.5 2.3 1.3 0.3]
 [6.6 2.9 4.6 1.3]
 [5.5 2.6 4.4 1.2]
 [5.3 3.7 1.5 0.2]]


## 3 · Vistas, copias y `strides`

In [22]:
big = np.arange(1e7)          # ~80 MB
print(big.nbytes)
view = big[::10]  
print(view)# slicing, vista
copy = big[::10].copy()        # copia
print("Memoria view:", view.nbytes/1e6, "MB")
print("Memoria copy:", copy.nbytes/1e6, "MB")

80000000
[0.00000e+00 1.00000e+01 2.00000e+01 ... 9.99997e+06 9.99998e+06
 9.99999e+06]
Memoria view: 8.0 MB
Memoria copy: 8.0 MB


### 📝 Ejercicio 5  
Usa `%timeit` para comparar la velocidad de `big.ravel()` versus `big.flatten()`.  
Explica por qué una es más rápida que la otra.

In [3]:
import numpy as np

# Crear un array grande para las pruebas
big = np.arange(1e7)  # ~80 MB

# Comparar tiempos de ejecución
%timeit big.ravel()
%timeit big.flatten()

80.4 ns ± 0.398 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
13.9 ms ± 347 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


##### Explicación de la diferencia de velocidad
###### La diferencia de velocidad se debe a cómo cada método maneja la memoria:  
#### ravel() 
Devuelve una vista del array original siempre que sea posible. Esto significa que no copia los datos, solo crea una nueva forma de acceder a los mismos datos en memoria (operación O(1)).  
#### flatten() 
Siempre crea una copia completa de los datos en un nuevo array contiguo en memoria (operación O(n)).  
Por lo tanto, ravel() es significativamente más rápido porque evita la costosa operación de copiar millones de elementos. Esto es especialmente notable en arrays grandes como el del ejemplo.

In [4]:
# Verificar si comparte memoria con el original
view = big.ravel()
copy = big.flatten()

print("ravel() comparte memoria:", view.base is big)  # Normalmente True
print("flatten() comparte memoria:", copy.base is big)  # Siempre False

ravel() comparte memoria: True
flatten() comparte memoria: False


## 4 · Conclusiones  
* El **broadcasting** permite operaciones vectorizadas sin expandir datos explícitamente.  
* Funciones como `reshape` y `split` reordenan vistas con costo O(1).  
* Verificar siempre si una operación produce **vista** o **copia** (`a.base is None`).  

### Lecturas recomendadas
* Johansson, *Numerical Python* (2024) — Cap. 2 y 3  
* Lott & Phillips, *Python OOP* — Apéndice sobre NumPy  
* Documentación oficial: https://numpy.org/doc/stable
